# Temporal Dynamics of Campus Placement Ecosystems: A Multi-Dimensional Visual Analytics Framework

### By:
### Nehan Ahmad - PES1UG23AM184
### Prateek P - PES1UG23AM211
### Noel Jose - PES1UG23AM197

In [ ]:
# Install required packages if not present
# !pip install pandas numpy matplotlib seaborn plotly scipy scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import json
import re
import warnings
from pathlib import Path
from typing import Dict, List, Optional
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from IPython.display import display, HTML, Markdown

# Configuration
warnings.filterwarnings('ignore')
plt.rcParams['figure.facecolor'] = 'white'
pd.set_option('display.max_columns', None)

# Create output directory
Path("processed_data").mkdir(exist_ok=True)

print("✅ Environment Setup Complete")